In [2]:
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from Handout_MNIST import MNIST

In [45]:
#Downloading digit dataset from load_digits
from sklearn.datasets import load_digits
import numpy as np

features, labels = load_digits(return_X_y=True)

test_features = features[-round(len(features)/5):]
train_features = features[:-round(len(features)/5)]
test_labels = labels[-round(len(labels)/5):]
train_labels = labels[:-round(len(labels)/5)]


print(f"Train data: {train_features.shape}")
print(f"Test data: {test_features.shape}")

Train data: (1438, 64)
Test data: (359, 64)


In [23]:
def get_gausian_distrubution_of_pixel_in_digit(threshold=10):
    # get the probability of each pixel being in a digit
    # (for each digit)
    digit_pixel_probabilities = []
    for digit in range(10):
        # get the indices of the pixels that belong to the digit
        digit_indices = np.where(train_labels == digit)
        
        
        
        for feature_set in train_features[digit_indices]:
            for pixel_index,pixel in enumerate(feature_set):
                if pixel > threshold:
                    refined_fetures[pixel_index] += 1

        digit_pixel_probabilities.append(refined_fetures/len(digit_indices[0]))
    return np.array(digit_pixel_probabilities)


def get_probability_of_number():
    return np.array([len(i)/len(train_labels) for i in [np.where(train_labels == i)[0] for i in range(10)]])

In [77]:
# Generationg the gaussian distribution of each pixel and number
gaussians_per_digit_and_pixel = []
for digit in range(10):
    # get the indices of the pixels that belong to the digit
    digit_indices = np.where(train_labels == digit)
    
    # Create mean and standard deviation for each pixel and each digit
    gaussian_means = [np.mean(train_features[digit_indices][:,pixel]+1) for pixel in range(train_features[digit_indices].shape[1])]
    gaussian_std = [np.std(train_features[digit_indices][:,pixel]+1) for pixel in range(train_features[digit_indices].shape[1])]
    
    # Merge into a single array
    gaussians_per_digit_and_pixel.append((gaussian_means,gaussian_std))

#print(gaussians_per_digit_and_pixel[0])

In [101]:
from scipy.stats import norm

def predict(feat, threshold=10):
    
    probability_of_number = get_probability_of_number()
    probabilities = []
    for i in range(10):
        
        log_sum = 0
        for index,j in enumerate(feat):
            norm_vals = (gaussians_per_digit_and_pixel[i][0][index],gaussians_per_digit_and_pixel[i][1][index])
            likleyhood = norm.pdf(j,norm_vals[0],norm_vals[1])
            if(j > threshold):
                log_sum += np.log(likleyhood)
        probabilities.append(log_sum+np.log(probability_of_number[i]))
    return np.argmax(probabilities)

predict(test_features[1])

/tmp/ipykernel_126016/2741543580.py:14: RuntimeWarning: divide by zero encountered in log
  log_sum += np.log(likleyhood)


4

In [102]:
# Printing metics
y_pred = []
for sample in test_features:
    y_pred.append(predict(sample))
        
print("Classification report SKLearn GNB:\n%s\n"
  % (metrics.classification_report(test_labels, y_pred)))
print(f"Confusion matrix:\n {metrics.confusion_matrix(test_labels, y_pred)}")

/tmp/ipykernel_126016/2741543580.py:14: RuntimeWarning: divide by zero encountered in log
  log_sum += np.log(likleyhood)


Classification report SKLearn GNB:
              precision    recall  f1-score   support

           0       0.91      0.86      0.88        35
           1       0.79      0.61      0.69        36
           2       1.00      0.32      0.49        34
           3       0.81      0.46      0.59        37
           4       0.75      0.89      0.81        37
           5       0.92      0.62      0.74        37
           6       1.00      0.30      0.46        37
           7       0.34      0.72      0.46        36
           8       0.40      0.82      0.54        33
           9       0.55      0.62      0.58        37

    accuracy                           0.62       359
   macro avg       0.75      0.62      0.62       359
weighted avg       0.75      0.62      0.63       359


Confusion matrix:
 [[30  0  0  0  3  0  0  0  2  0]
 [ 0 22  0  0  0  0  0  1  5  8]
 [ 0  0 11  0  4  0  0 19  0  0]
 [ 0  0  0 17  0  0  0  6 14  0]
 [ 0  0  0  0 33  0  0  1  3  0]
 [ 0  1  0  0  1 23  